# D&D AI Model Training with Unsloth and Gemma3N

This notebook fine-tunes a Gemma3N model specifically for D&D gameplay using Unsloth for efficient training and exports to GGUF format for local deployment with Cactus AI.

## Overview

- **Base Model**: Gemma3N (2B/4B variants)
- **Training Framework**: Unsloth for 2x faster training with 70% less VRAM
- **Output Format**: GGUF for Ollama/llama.cpp compatibility
- **Target Application**: Cactus AI integration for D&D gameplay

## Requirements

- Python 3.9-3.11
- 16GB+ RAM (32GB recommended)
- GPU with 8GB+ VRAM (16GB recommended) or Apple Silicon Mac
- 50GB+ free disk space

## 1. Environment Setup and Validation

In [ ]:
import sys
import os
import platform
import torch
import psutil
from pathlib import Path

# System information
print(f"Python Version: {sys.version}")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Architecture: {platform.machine()}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"MPS Available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False}")
print(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name()}")
    print(f"CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")

# Set device
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
    
print(f"\nUsing device: {device}")

: 

## 2. Install Dependencies

Install Unsloth and required packages. This will take a few minutes.

In [ ]:
# Install Unsloth and dependencies
!pip install unsloth[colab-new] --quiet

# Install additional required packages
!pip install datasets transformers accelerate bitsandbytes --quiet

# For GGUF export
!pip install llama-cpp-python --quiet

print("Dependencies installed successfully!")

## 3. Import Required Libraries

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments
from trl import SFTTrainer
import json
import pandas as pd
from datetime import datetime
import re
import numpy as np
from pathlib import Path
import markdown
from typing import List, Dict, Tuple, Optional

print("All libraries imported successfully!")

## 4. Configuration and Hyperparameters

In [ ]:
# Training configuration
CONFIG = {
    # Model settings
    "model_name": "unsloth/gemma-3-2b-it",  # Can also use "unsloth/gemma-3-4b-it"
    "max_seq_length": 2048,  # Max context length
    "load_in_4bit": True,    # 4-bit quantization for memory efficiency
    "dtype": None,           # Auto-detect best dtype
    
    # LoRA settings
    "r": 16,                 # LoRA rank
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", 
                       "gate_proj", "up_proj", "down_proj"],
    "lora_alpha": 16,
    "lora_dropout": 0,       # No dropout for better convergence
    "bias": "none",
    "use_gradient_checkpointing": "unsloth",
    "random_state": 42,
    "use_rslora": False,     # Set to True for large datasets
    "loftq_config": None,
    
    # Training hyperparameters
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "warmup_steps": 5,
    "max_steps": 60,         # Adjust based on dataset size
    "learning_rate": 2e-4,
    "fp16": not torch.cuda.is_bf16_supported(),
    "bf16": torch.cuda.is_bf16_supported(),
    "logging_steps": 1,
    "optim": "adamw_8bit",
    "weight_decay": 0.01,
    "lr_scheduler_type": "linear",
    "seed": 42,
    
    # Output settings
    "output_dir": "./trained_models",
    "model_save_name": "dnd_gemma3_2b",
    "gguf_quantization": "q4_k_m",
    
    # D&D specific settings
    "system_message": "You are a Dungeon Master assistant for D&D 5e. You help with gameplay, rules, and story generation. Use tool calls when needed for game mechanics.",
    "tool_format": "[{tool_name}: {arguments}]",
    "supported_tools": ["roll", "health", "inventory", "spellcast", "check", "save"]
}

print("Configuration loaded:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 5. Load Base Model and Tokenizer

In [ ]:
# Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=CONFIG["model_name"],
    max_seq_length=CONFIG["max_seq_length"],
    dtype=CONFIG["dtype"],
    load_in_4bit=CONFIG["load_in_4bit"],
)

print(f"Base model loaded: {CONFIG['model_name']}")
print(f"Model parameters: {model.num_parameters():,}")
print(f"Tokenizer vocab size: {len(tokenizer)}")

# Configure LoRA for fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=CONFIG["r"],
    target_modules=CONFIG["target_modules"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    bias=CONFIG["bias"],
    use_gradient_checkpointing=CONFIG["use_gradient_checkpointing"],
    random_state=CONFIG["random_state"],
    use_rslora=CONFIG["use_rslora"],
    loftq_config=CONFIG["loftq_config"],
)

print("LoRA adapters configured successfully!")
print(f"Trainable parameters: {model.get_nb_trainable_parameters()[0]:,}")
print(f"Training efficiency: {model.get_nb_trainable_parameters()[1]:.2%}")